In [3]:

import pickle
import cv2
import os
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from torchvision.datasets import ImageFolder

In [4]:
print(torch.__version__)
print(torch.cuda.is_available())

2.2.0+cu121
True


In [5]:
# # 把官網下載的batch轉成圖片
# # 要先把batches.meta跟readme.html拿走
# def unpickle_cifar10(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict[b'labels'], dict[b'filenames'], dict[b'data']

# def unpickle_cifar100(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict[b'fine_labels'], dict[b'filenames'], dict[b'data']


# with open("cifar-100-python/meta", 'rb') as file:
#     matadata = pickle.load(file, encoding='bytes')
#     print(dict)

# labelName = matadata[b'fine_label_names']
# for i in range(len(labelName)):
#     labelName[i] = labelName[i].decode('utf-8')


# for path in os.listdir('cifar-100-python'):
#     print("Extract: ", path)
#     if path == "meta":
#         continue
#     labels,names, datas = unpickle_cifar100(f'cifar-100-python/{path}')
#     for label, name, data in zip(labels,names,datas):
#         try:
#             os.makedirs(f'cifar_100_pic/train/{labelName[label]}')
#             os.makedirs(f'cifar_100_pic/test/{labelName[label]}')
#         except:
#             pass
            
#         R = data[:1024]
#         G = data[1024:2048]
#         B = data[2048:3072]
        
#         img,tmp = [],[]
#         for cnt,(r,g,b) in enumerate(zip(R,G,B),1):
#             tmp.append([b, g, r])
#             if cnt % 32 == 0:
#                 img.append(tmp)
#                 tmp = []
              
#         if path == 'test':
#             cv2.imwrite(f'cifar_100_pic/test/{labelName[label]}/{str(name)[2:-1]}', np.array(img))
#         else:
#             cv2.imwrite(f'cifar_100_pic/train/{labelName[label]}/{str(name)[2:-1]}', np.array(img))


In [6]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(ResidualBlock, self).__init__()
        # Convolution -> Batch Normalization -> ReLu
        self.conv1 = torch.nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(num_features=out_channel)
        self.conv2 = torch.nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(num_features=out_channel)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = torch.nn.functional.relu(out)
        out = self.conv2(out)
        out += residual # 加上原本的 -> Residual
        out = torch.nn.functional.relu(out)
        return out

In [7]:
# 跟ResidualBlock一樣，但是要降tensor的大小 -> stride要改
class DownBlock(torch.nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(DownBlock, self).__init__()
        # 兩層convolution
        self.conv1 = torch.nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride[0], padding=1)
        self.bn1 = torch.nn.BatchNorm2d(num_features=out_channel)
        self.conv2 = torch.nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=stride[1], padding=1)
        self.bn2 = torch.nn.BatchNorm2d(num_features=out_channel)
        self.adjustChannel = torch.nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=1, stride=stride[0])
    
    def forward(self, x):
        residual = x
        residual = self.adjustChannel(residual)
        out = self.conv1(x)
        out = self.bn1(out)
        out = torch.nn.functional.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual # 加上原本的 -> Residual
        out = torch.nn.functional.relu(out)
        return out

In [8]:

class ResNet18(torch.nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        # conv1, conv2, conv3... 對照ResNet18_architecture.png命名
        
        # channel x W x H
        # output size = (W - kernel_size + 2 *padding) / stride

        # input: 3x32x32
        # output: 64x16x16
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=64, stride=2, kernel_size=7, padding=3)

        # input: 64x16x16
        # output: 64x8x8
        self.maxPool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # input: 64x8x8
        # output: 64x8x8
        self.conv2 = torch.nn.Sequential(ResidualBlock(64, 64, 1),
                                         ResidualBlock(64, 64, 1))
        
        # input: 64x8x8
        # output: 128x4x4
        self.conv3 = torch.nn.Sequential(DownBlock(64, 128, [2, 1]),
                                         ResidualBlock(128, 128, 1))
        
        # input: 128x4x4
        # output: 256x2x2
        self.conv4 = torch.nn.Sequential(DownBlock(128, 256, [2, 1]),
                                         ResidualBlock(256, 256, 1))
        
        # input: 256x2x2
        # output: 512x1x1
        self.conv5 = torch.nn.Sequential(DownBlock(256, 512, [2, 1]),
                                         ResidualBlock(512, 512, 1))
        
        # input: 512x1x1
        # output: 512x1x1
        self.averagePool = torch.nn.AdaptiveAvgPool2d(output_size=(1,1))

        # input: 512x1x1
        # output: 100
        self.fc = torch.nn.Linear(in_features=512, out_features=100)

    def forward(self, x):
        out = self.conv1(x)
        out = self.maxPool(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.averagePool(out)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out


In [9]:
def pil2tensor(img):
    transform = torchvision.transforms.ToTensor()
    result = transform(img)
    return result

def int2tensor(number):
    result = torch.tensor(int(number))
    return result

In [13]:
# main
cifar_train = ImageFolder(root="cifar_100_pic/train", transform=pil2tensor, target_transform=int2tensor)
cifar_test = ImageFolder(root="cifar_100_pic/test", transform=pil2tensor, target_transform=int2tensor)

trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=16,shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=16, shuffle=False, num_workers=2)

device = torch.device('cuda')
model = ResNet18().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    epoch_loss = 0.0
    data_loader = tqdm(trainloader, desc=f"Epoch {epoch+1}", leave=True)  # leave=False: 結束後進度條不會消失
    for idx, (x, label) in enumerate(data_loader):
        x, label = x.to(device), label.to(device)
        
        result = model(x)
        loss = criterion(result, label)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        data_loader.set_postfix({'Loss': epoch_loss / (idx + 1)})
    
    print(epoch, "loss:", loss.item())

Epoch 1: 100%|██████████| 3125/3125 [00:31<00:00, 100.15it/s, Loss=4.12]


0 loss: 3.67252779006958


Epoch 2: 100%|██████████| 3125/3125 [00:31<00:00, 99.45it/s, Loss=3.65] 


1 loss: 3.1448214054107666


Epoch 3: 100%|██████████| 3125/3125 [00:31<00:00, 99.36it/s, Loss=3.32] 


...

7 loss: 1.8519868850708008


Epoch 9: 100%|██████████| 3125/3125 [00:31<00:00, 98.79it/s, Loss=2.2]  


8 loss: 2.1384706497192383


Epoch 10: 100%|██████████| 3125/3125 [00:31<00:00, 98.96it/s, Loss=2.08] 

9 loss: 1.7690125703811646


In [16]:
print("Test the Model")
data_loader = tqdm(testloader, desc=f"test", leave=True)  # leave=False: 結束後進度條不會消失


with open("cifar-100-python/meta", 'rb') as file:
    matadata = pickle.load(file, encoding='bytes')
labelName = matadata[b'fine_label_names']
for i in range(len(labelName)):
    labelName[i] = labelName[i].decode('utf-8')


class_correct = list(0. for i in range(len(labelName)))
class_total = list(0. for i in range(len(labelName)))
total = 0
total_correct = 0

with torch.no_grad():
    for data in data_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(16):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

            total_correct += c[i].item()
            total +=1


# classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print('Accuracy of the network : %d %%' % (100 * total_correct / total), "Number:", total)
print()
for i in range(len(labelName)):
    print('Accuracy of %5s : %2d %%' % (
        labelName[i], 100 * class_correct[i] / class_total[i]))



Test the Model


test:   0%|          | 0/625 [00:00<?, ?it/s]

<class 'dict'>


test: 100%|██████████| 625/625 [00:02<00:00, 226.81it/s]

Accuracy of the network : 13 % Number: 10000

Accuracy of apple : 19 %
Accuracy of aquarium_fish : 19 %
Accuracy of  baby : 10 %
Accuracy of  bear :  4 %
Accuracy of beaver :  2 %
Accuracy of   bed : 15 %
Accuracy of   bee : 16 %
Accuracy of beetle : 11 %
Accuracy of bicycle : 13 %
Accuracy of bottle : 22 %
Accuracy of  bowl :  8 %
Accuracy of   boy :  6 %
Accuracy of bridge : 20 %
Accuracy of   bus :  9 %
Accuracy of butterfly : 10 %
Accuracy of camel :  5 %
Accuracy of   can : 10 %
Accuracy of castle : 21 %
Accuracy of caterpillar :  6 %
Accuracy of cattle : 14 %
Accuracy of chair : 22 %
Accuracy of chimpanzee : 15 %
Accuracy of clock : 23 %
Accuracy of cloud : 18 %
Accuracy of cockroach : 35 %
Accuracy of couch : 16 %
Accuracy of  crab :  5 %
Accuracy of crocodile :  5 %
Accuracy of   cup : 12 %
Accuracy of dinosaur : 20 %
Accuracy of dolphin : 12 %
Accuracy of elephant : 16 %
Accuracy of flatfish : 10 %
Accuracy of forest : 11 %
Accuracy of   fox : 16 %
Accuracy of  girl :  7 %
Acc